<a href="https://colab.research.google.com/github/MohammadObeidat03/Accident-detection/blob/main/pytorch_tensorflow_resnet_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PyTorch – Full Fine-Tuning


In [1]:
import torch
import torch.nn as nn
from torchvision import models

def resnet_full_finetune(num_classes=10):
    model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

    # Replace final FC
    model.fc = nn.Linear(model.fc.in_features, num_classes)

    # Allow training of all layers
    for p in model.parameters():
        p.requires_grad = True

    return model


#PyTorch – Freeze + Add CNN + FC Head

In [2]:
import torch
import torch.nn as nn
from torchvision import models

class FrozenResNetWithHead(nn.Module):
    def __init__(self, num_classes=10):
        super().__init__()
        base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

        # Backbone until layer4 output
        self.backbone = nn.Sequential(*list(base.children())[:-2])

        # Freeze ResNet weights
        for p in self.backbone.parameters():
            p.requires_grad = False

        # New CNN + FC head
        self.head = nn.Sequential(
            nn.Conv2d(2048, 512, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d((1,1)),
            nn.Flatten(),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.backbone(x)
        x = self.head(x)
        return x


#TensorFlow – Full Fine-Tuning

In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

def resnet_full_finetune_tf(input_shape=(224,224,3), num_classes=10):
    base = ResNet50(include_top=False, weights="imagenet", input_shape=input_shape)
    base.trainable = True   # Full fine-tuning

    inputs = layers.Input(shape=input_shape)
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base(x, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    return model


#TensorFlow – Freeze + Add CNN + FC Head

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50

def resnet_frozen_with_head_tf(input_shape=(224,224,3), num_classes=10):
    base = ResNet50(include_top=False, weights="imagenet", input_shape=input_shape)
    base.trainable = False   # Freeze backbone

    inputs = layers.Input(shape=input_shape)
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base(x, training=False)
    x = layers.Conv2D(512, 3, padding="same", activation="relu")(x)
    x = layers.GlobalAveragePooling2D()(x)
    outputs = layers.Dense(num_classes, activation="softmax")(x)

    model = models.Model(inputs, outputs)
    return model
